# Projet final du Kit Big Data 2022

In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [76]:
import pandas as pd
import re
import os
import numpy as np

import time
import datetime
from unidecode import unidecode

import requests
from bs4 import BeautifulSoup

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

In [77]:
from src.utils import make_request, download_file, create_dir, remove_file, remove_dir
from src.utils import load_csv, save_csv, save_with_xlwings

In [78]:
DATA_DIR = 'data/'
DATA_RAW_DIR = DATA_DIR + 'raw/'
DATA_PROCESSED_DIR = DATA_DIR + 'processed/'

# remove_dir(DATA_RAW_DIR)
# remove_dir(DATA_PROCESSED_DIR)

create_dir(DATA_RAW_DIR)
create_dir(DATA_PROCESSED_DIR)

In [79]:
pd.set_option('display.max_columns', None)

# Partie 1 - Acquisition et chargement des données

### A) Acquérir l'ensemble des fichiers Excel des classements via la page de classement

In [80]:
def retrieve_date_rankings_list(url="https://www.vendeeglobe.org/fr/classement", verbose=False):
    """ Retrieve the list of rankings date 
    Args:
        url (String, optional): web page url
    Returns:
        date_rankings_list (List of strings): list of rankings date 
    """
    # Make request
    req = make_request(url, verbose=verbose)

    # Retrieve whole content
    soup = BeautifulSoup(req.content)

    # Retrieve specific information
    soup_info = soup.find('select', class_='form__input m--select onsubmit-rank')

    date_rankings = soup_info.find_all('option')
    date_rankings_list = [date_ranking.get('value') for date_ranking in date_rankings[1:]]
    
    # Remove duplicates
    date_rankings_list = list(set(date_rankings_list))
    return date_rankings_list

In [81]:
date_rankings_list = retrieve_date_rankings_list(verbose=True)
print(f"{len(date_rankings_list)} date rankings retrieved")

Request url https://www.vendeeglobe.org/fr/classement
703 date rankings retrieved


In [82]:
def download_rankings_results(page_date_rankings, dir_dst='', domain='https://www.vendeeglobe.org', page='/download-race-data/vendeeglobe_{page_id}.xlsx', N=100, verbose=False):
    """ Download rankings results
    Args:
        page_date_rankings (List of strings): list of rankings date 
        dir_dst (String, optional): directory destination path
        domain (String, optional): domain name
        page (String, optional): page name
        N (Integer, optional): number of elements to display if verbose is True
    Returns:
        None
    """
    for page_no, page_date_ranking in enumerate(page_date_rankings):
        
        # Create url and filename dest path
        url = domain + page.format(page_id=page_date_ranking)
        filename = dir_dst + os.path.basename(page.format(page_id=page_date_ranking))

        if verbose:
            if (page_no+1) % N == 0:
                print(f"[{page_no+1}/{len(page_date_rankings)}] downloading pages..")
                
        # Download xlsx file
        download_file(url, filename)
        
    if verbose:
        if len(page_date_rankings) % N:
            print(f"[{page_no+1}/{len(page_date_rankings)}] downloading pages..")

In [83]:
ranking_dir_dst = DATA_RAW_DIR + 'ranking/'

# Create folder if does not exist 
create_dir(ranking_dir_dst)

# Check if directory is empty and download rankings results
if not os.listdir(ranking_dir_dst):
    print(f"Directory '{ranking_dir_dst}' is empty. Downloading rankings results..")
    download_rankings_results(date_rankings_list, dir_dst=ranking_dir_dst, verbose=True)
    print(f"Downloading done.")
else:
    print(f"Data already dowloaded in '{ranking_dir_dst}' directory")

Data already dowloaded in 'data/raw/ranking/' directory


In [84]:
month_name2month_no = dict(
    janvier=1,
    fevrier=2,
    mars=3,
    avril=4,
    mai=5,
    juin=6,
    juillet=7,
    aout=8,
    septembre=9,
    octobre=10,
    novembre=11,
    decembre=12
)


def get_skipper_race_status(skipper_rank):
    """ Identify skipper race status (whethere skipper is still in race or not)
    Args:
        df (DataFrame): data with multindex in columns
    Returns:
        skipper_in_race (Boolean): skipper race status
    """
    if (isinstance(skipper_rank, str)):
        if skipper_rank.strip().isnumeric():
            skipper_in_race = 'IN RACE'   
        else:
            if 'ARV' in skipper_rank:
                skipper_in_race = 'NOT IN RACE'
            elif 'RET' in skipper_rank:
                skipper_in_race = 'IN RACE'
            else:
                skipper_in_race = np.nan            
    else:
        skipper_in_race = np.nan
    return skipper_in_race



def extract_ranking_datetime(df, verbose=False):
    """ Extract ranking datetime
    Args:
        df (DataFrame): data
    Returns:
        ranking_datetime (Datetime) ranking datetime
    """
    # Retrieve datetime
    ranking_datetime = ' '.join(df.iloc[1][1].split()[3:-1])
    if verbose:
        print(f">>> ranking_datetime: {ranking_datetime}")

    # Divide datetime into elements
    ranking_datetime = re.sub(' à ', ' ', ranking_datetime)
    day, month_name, year, fr_time = tuple(ranking_datetime.split())
    if verbose:
        print(f"day month_name year fr_time: {day} {month_name} {year} {fr_time}")

    # Convert french time to english time (18h00 -> 06:00PM)
    fr_hour, fr_min = tuple(fr_time.split('h'))
    # en_time = "%02d:%02d" % (int(fr_hour), int(fr_min)) 
    # t = time.strptime(en_time, "%H:%M")
    # en_time = time.strftime("%I:%M %p", t)
    # if verbose:
    #     print(f"fr_time ({fr_time}) -> en_time ({en_time})")
    # ranking_time = datetime.time(int(str(en_time).split(':')[0]), int(fr_min))
    ranking_time = datetime.time(int(fr_hour), int(fr_min))
    if verbose:
        print(f">>> ranking_time: {ranking_time}")

    # Convert string date to date (03 mars 2021 -> 2022-03-01)
    ranking_date = datetime.datetime(int(year), month_name2month_no[unidecode(month_name).lower()], int(day))
    
    if verbose:
        print(f">>> ranking_date: {ranking_date}")

    # Create ranking_datetime using ranking_date and ranking_time
    ranking_datetime = datetime.datetime.combine(ranking_date, ranking_time)
    if verbose:
        print(f">>> ranking_datetime: {ranking_datetime}")
        
    return ranking_datetime



def load_skippers_ranking(ranking_dir_dst, N=100, verbose=False):
    """ Load skippers ranking from xslx files into dataframes
    Args:
        ranking_dir_dst (String, optional): directory destination path
        N (Integer, optional): number of elements to display if verbose is True
    Returns:
        df_skippers_not_in_race (DataFrame): data with skippers whos have finished the race
        df_skippers_in_race (DataFrame): data with skippers still in race/has abandonned the race
    """
    df_skippers_not_in_race = pd.DataFrame()
    df_skippers_in_race = pd.DataFrame()

    ranking_xslx_paths = [ranking_dir_dst+filename for filename in os.listdir(ranking_dir_dst)]#[-50:]
    for file_no, filename in enumerate(ranking_xslx_paths):
        if verbose:
            if (file_no+1) % N == 0:
                print(f"[{file_no+1}/{len(ranking_xslx_paths)}] loading and sorting skippers..")
                
        # Read excel file using xlwings
        uuid_dir_dst = DATA_RAW_DIR + 'uuid/'
        create_dir(uuid_dir_dst)
        filename_uuid = save_with_xlwings(filename, dir_dst=uuid_dir_dst)
        df_skippers = pd.read_excel(filename_uuid) 

        # Insert ranking date
        df_skippers.insert(0, 'ranking_datetime', extract_ranking_datetime(df_skippers))
        
        # Map skippers race status (0: skippers has finished the race, 1: skippers is still/has abandonned in race, NaN: other)
        df_skippers = df_skippers.rename(columns={'Unnamed: 0':'skipper_status'})
        df_skippers['skipper_status'] = df_skippers[list(df_skippers.columns)[2]].apply(lambda x: get_skipper_race_status(x))

        # Add skippers to dfs
        df_skippers_not_in_race = pd.concat([df_skippers_not_in_race, df_skippers[df_skippers['skipper_status'] == 'NOT IN RACE']], ignore_index=True)
        df_skippers_in_race = pd.concat([df_skippers_in_race, df_skippers[df_skippers['skipper_status'] == 'IN RACE']], ignore_index=True)
        
        # Remove uuid file
        remove_file(filename_uuid)
        
    if verbose:
        if len(ranking_xslx_paths) % N:
            print(f"[{file_no+1}/{len(ranking_xslx_paths)}] loading and sorting skippers..")
    
    return df_skippers_not_in_race, df_skippers_in_race

In [85]:
skippers_race_status_raw_dir_dst = DATA_RAW_DIR + 'skippers_race_status/'
filename_not_in_race = skippers_race_status_raw_dir_dst + 'skippers_not_in_race.csv'
filename_in_race = skippers_race_status_raw_dir_dst + 'skippers_in_race.csv'
    
# Create folder if does not exist 
create_dir(skippers_race_status_raw_dir_dst)

# Check if directory is empty and download rankings results
if not os.listdir(skippers_race_status_raw_dir_dst):
    print(f"Directory '{skippers_race_status_raw_dir_dst}' is empty. Loading skippers race status..")
    df_skippers_not_in_race_raw, df_skippers_in_race_raw = load_skippers_ranking(ranking_dir_dst, verbose=True)
    print(f"Loading done.")    
    
    # Save data
    save_csv(df_skippers_not_in_race_raw, csv_path=filename_not_in_race, verbose=True)
    save_csv(df_skippers_in_race_raw, csv_path=filename_in_race, verbose=True)
else:
    df_skippers_not_in_race_raw = load_csv(filename_not_in_race)
    df_skippers_in_race_raw = load_csv(filename_in_race)
    print(f"Data already in '{skippers_race_status_raw_dir_dst}' directory")
    
print(f">>> Total: {df_skippers_not_in_race_raw.shape[0]+df_skippers_in_race_raw.shape[0]} skippers:\n\t- {df_skippers_not_in_race_raw.shape[0]} skippers have finished the race\n\t- {df_skippers_in_race_raw.shape[0]} skippers still in race or have abandonned the race")

Data already in 'data/raw/skippers_race_status/' directory
>>> Total: 23161 skippers:
	- 3931 skippers have finished the race
	- 19230 skippers still in race or have abandonned the race


In [86]:
df_skippers_not_in_race_raw

,ranking_datetime,skipper_status,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2021-01-27 18:00:00,NOT IN RACE,1\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 21:35:47 FR,80j 06h 15min 47s\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6 kts,24355.0 nm,119.6 %,15.1 kts,29135.0 nm
1,2021-01-27 22:00:00,NOT IN RACE,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 05:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
2,2021-01-27 22:00:00,NOT IN RACE,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 21:35:47 FR,80j 06h 15min 47s\n,NaN,NaN,NaN,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
3,2021-01-27 22:00:00,NOT IN RACE,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,NaN,NaN,28/01/2021 01:45:12 FR,80j 10h 25min 12s\n,NaN,NaN,NaN,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
4,2021-01-28 05:00:00,NOT IN RACE,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 05:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,2021-03-05 09:00:00,NOT IN RACE,21\nARV,\nFRA 83,Clément Giraud\nCompagnie du lit - Jiliti,NaN,NaN,NaN,16/02/2021 10:28:31 FR,99j 20h 08min 31s\n,NaN,NaN,NaN,NaN,19j 16h 23min 45s,NaN,2j 13h 41min 28s,10.2 kts,24365.7 nm,115.5 %,11.7 kts,28137.8 nm
3927,2021-03-05 09:00:00,NOT IN RACE,22\nARV,\nFRA 50,Miranda Merron\nCampagne de France,NaN,NaN,NaN,17/02/2021 23:16:51 FR,101j 08h 56min 51s\n,NaN,NaN,NaN,NaN,21j 05h 12min 05s,NaN,1j 12h 48min 20s,10.0 kts,24365.7 nm,113.5 %,11.4 kts,27656.2 nm
3928,2021-03-05 09:00:00,NOT IN RACE,23\nARV,\nFRA 71,Manuel Cousin\nGroupe Sétin,NaN,NaN,NaN,20/02/2021 08:35:40 FR,103j 18h 15min 40s\n,NaN,NaN,NaN,NaN,23j 14h 30min 54s,NaN,2j 09h 18min 49s,9.8 kts,24365.7 nm,119.5 %,11.7 kts,29115.7 nm
3929,2021-03-05 09:00:00,NOT IN RACE,24\nARV,\nFRA 72,Alexia Barrier\nTSE - 4myplanet,NaN,NaN,NaN,28/02/2021 07:23:44 FR,111j 17h 03min 44s\n,NaN,NaN,NaN,NaN,31j 13h 18min 58s,NaN,7j 22h 48min 04s,9.1 kts,24365.7 nm,115.6 %,10.5 kts,28170.7 nm


In [87]:
df_skippers_in_race_raw

,ranking_datetime,skipper_status,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,2020-11-08 16:00:00,IN RACE,1,\nFRA 18,Louis Burton\nBureau Vallée 2,15:30 FR\n,46°24.46'N,01°50.48'W,241°,17.7 kts,17.5 kts,0.3 nm,357°,0.0 kts,0.0 kts,2788.0 nm,201°,0.3 kts,0.3 kts,6.1 nm,24293.9 nm,0.0 nm
1,2020-11-08 16:00:00,IN RACE,2,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,15:31 FR\n1min,46°24.34'N,01°49.82'W,241°,11.1 kts,10.9 kts,0.4 nm,357°,0.0 kts,0.0 kts,2787.9 nm,196°,0.3 kts,0.2 kts,6.0 nm,24294.2 nm,0.4 nm
2,2020-11-08 16:00:00,IN RACE,3,\nFRA 8,Jérémie Beyou\nCharal,15:30 FR\n,46°24.91'N,01°49.99'W,244°,15.5 kts,15.5 kts,0.5 nm,357°,0.0 kts,0.0 kts,2788.5 nm,199°,0.2 kts,0.2 kts,5.5 nm,24294.3 nm,0.5 nm
3,2020-11-08 16:00:00,IN RACE,4,\nFRA 59,Thomas Ruyant\nLinkedOut,15:30 FR\n,46°24.71'N,01°49.68'W,244°,13.2 kts,13.1 kts,0.7 nm,357°,0.0 kts,0.0 kts,2788.3 nm,196°,0.2 kts,0.2 kts,5.6 nm,24294.5 nm,0.6 nm
4,2020-11-08 16:00:00,IN RACE,5,\nFRA 53,Maxime Sorel\nV And B Mayenne,15:30 FR\n,46°24.59'N,01°49.56'W,246°,10.9 kts,10.9 kts,0.2 nm,357°,0.0 kts,0.0 kts,2788.1 nm,195°,0.8 kts,0.7 kts,5.8 nm,24294.5 nm,0.6 nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19225,2021-03-05 09:00:00,IN RACE,RET,\nFRA 109,Samantha Davies\nInitiatives - Coeur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19226,2021-03-05 09:00:00,IN RACE,RET,\nFRA 4,Sébastien Simon\nARKEA PAPREC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19227,2021-03-05 09:00:00,IN RACE,RET,\nGBR 99,Alex Thomson\nHUGO BOSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19228,2021-03-05 09:00:00,IN RACE,RET,\nFRA 85,Kevin Escoffier\nPRB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### B) Acquérir les caractéristiques des bateaux

In [88]:
def retrieve_boats_characteristics(url="https://www.vendeeglobe.org/fr/glossaire", verbose=False):
    """ Retrieve the list of rankings date 
    Args:
        url (String, optional): web page url
    Returns:
        df_boats_characteristics (DataFrame): boats characteristics
    """
    boats_characteristics_list = []

    # Make request
    req = make_request(url, verbose=verbose)

    # Retrieve whole content
    soup = BeautifulSoup(req.content)

    # Retrieve specific information
    current_boats_info = soup.find_all('div', class_='boats-list__infos')
    boats_name = [current_boat_info.find('h3').text.strip() for current_boat_info in current_boats_info]
    skippers_name = [current_boat_info.find('span').text.strip() for current_boat_info in current_boats_info]
    
    soup_info = soup.find_all('div', class_='boats-list__popup-infos')
    for boat_no, soup_boat_info in enumerate(soup_info):
        boat_info = soup_boat_info.find_all('li')

        # Retrieve the boat characteristics
        boat_characteristics = [tuple(characteristics.text.split(' : ', maxsplit=1)) for characteristics in boat_info]
        boat_characteristics = dict(map(reversed, boat_characteristics))
        boat_characteristics = {v: k for k, v in boat_characteristics.items()}
        dct = dict(skipper_name=skippers_name[boat_no], boat_name=boats_name[boat_no])
        dct.update(boat_characteristics)
        boats_characteristics_list.append(dct)

    df_boats_characteristics = pd.DataFrame(boats_characteristics_list)
    return df_boats_characteristics

In [89]:
boat_raw_dir_dst = DATA_RAW_DIR + 'boat/'
filename = boat_raw_dir_dst + 'boats_characteristics.csv'

# Create folder if does not exist 
create_dir(boat_raw_dir_dst)

# Check if directory is empty and retrieve boats characteristics
if not os.listdir(boat_raw_dir_dst):
    print(f"Directory '{boat_raw_dir_dst}' is empty. Scraping boats characteristics..")
    df_boats_characteristics_raw = retrieve_boats_characteristics(verbose=True)
    print(f"Scraping done.\n")
    
    # Save data
    save_csv(df_boats_characteristics_raw, csv_path=filename, verbose=True)
    
else:
    df_boats_characteristics_raw = load_csv(filename)
    
df_boats_characteristics_raw

,skipper_name,boat_name,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,Fabrice AMEDEO,NEWREST - ART & FENÊTRES,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,Romain ATTANASIO,PURE - Best Western®,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,Alexia BARRIER,TSE - 4MYPLANET,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,Yannick BESTAVEN,Maître CoQ IV,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,Jérémie BEYOU,CHARAL,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,Arnaud BOISSIÈRES,LA MIE CÂLINE - ARTISANS ARTIPÔLE,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,Louis BURTON,BUREAU VALLEE 2,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,Didac COSTA,ONE PLANET ONE OCEAN,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,Manuel COUSIN,GROUPE SÉTIN,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,Clarisse CREMER,BANQUE POPULAIRE X,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2


# Partie 2 - Préparation des données

### A) Préparation des données relatives aux classements.

In [90]:
def get_skipper_cols(skipper_in_race):
    """ Get skipper columns in function of his race status
    Args:
        skipper_in_race (Boolean): skipper race status
    Returns:
        cols (List of strings): columns related to skipper race status
    """
    skipper_class2cols = {
        0: [
            'ranking_datetime', 'skipper_status', 
            'rank', 'nationality & sail', 'skipper & crew',
            'arrival_date', 'race_time', 
            'gaps_to_first', 'gaps_to_previous', 
            'over_ortho_speed', 'over_ortho_distance', 'play',
            'over_ground_speed', 'over_ground_distance' 
        ],
        1: [
            'ranking_datetime', 'skipper_status', 
            'rank', 'nationality & sail', 'skipper & crew', 'hour_fr', 'latitude', 'longitude',
            'since30min_heading', 'since30min_speed', 'since30min_vmg', 'since30min_distance', 
            'sincelastreport_heading', 'sincelastreport_speed', 'sincelastreport_vmg', 'sincelastreport_distance',
            'since24hours_heading', 'since24hours_speed', 'since24hours_vmg', 'since24hours_distance',
            'dtf', 'dtl'
        ]
    }
    cols = skipper_class2cols[int(skipper_in_race)]
    return cols


def dms2dec(deg, mn, sec):
    """ Convert degree, minutes and secondes into decimal number 
    Args:
    Returns:
    """
    dec = deg + mn/60 + sec/3600
    return dec


def preprocess_skippers_ranking(df_skippers_not_in_race, df_skippers_in_race, verbose=False):
    """ Load skippers ranking from xslx files into dataframes
    Args:
        df_skippers_not_in_race (DataFrame): data with skippers whos have finished the race
        df_skippers_in_race (DataFrame): data with skippers still in race/has abandonned the race
    Returns:
        df_not_in (DataFrame): preprocessed dataframe
        df_in (DataFrame): preprocessed dataframe
    """
    df_not_in = df_skippers_not_in_race.copy()
    df_in = df_skippers_in_race.copy()
    
    # Drop columns with only NaN values
    df_not_in = df_not_in.dropna(axis=1, how='all')
    df_in = df_in.dropna(axis=1, how='all')

    # Rename columns
    df_not_in.columns = get_skipper_cols(skipper_in_race=False)
    df_in.columns = get_skipper_cols(skipper_in_race=True)
    
    # Clean data (df_not_in)
    df_not_in['rank'] = df_not_in['rank'].apply(lambda x: x.split()[0])
    
    df_not_in['nationality'] = df_not_in['nationality & sail'].apply(lambda x: x.split()[0])
    df_not_in['sail'] = df_not_in['nationality & sail'].apply(lambda x: x.split()[1])
    
    df_not_in['skipper_firstname'] = df_not_in['skipper & crew'].apply(lambda x: x.split('\n')[0].split()[0])
    df_not_in['skipper_lastname'] = df_not_in['skipper & crew'].apply(lambda x: ' '.join(x.split('\n')[0].split()[1:]).upper().replace(' ', ''))
    df_not_in['crew_name'] = df_not_in['skipper & crew'].apply(lambda x: x.split('\n')[1])
    
    df_not_in['race_time_gap'] = df_not_in['race_time'].apply(lambda x: x.split('\n')[-1])
    df_not_in['race_time'] = df_not_in['race_time'].apply(lambda x: x.split('\n')[0])
    df_not_in['arrival_date'] = df_not_in['arrival_date'].apply(lambda x: datetime.datetime.strptime(x.replace(' FR', ''), '%d/%m/%Y %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S'))
    
    df_not_in['over_ortho_speed'] = df_not_in['over_ortho_speed'].apply(lambda x: float(x.replace(' kts', '')))
    df_not_in['over_ortho_distance'] = df_not_in['over_ortho_distance'].apply(lambda x: float(x.replace(' nm', '')))
    df_not_in['over_ground_speed'] = df_not_in['over_ground_speed'].apply(lambda x: float(x.replace(' kts', '')))
    df_not_in['over_ground_distance'] = df_not_in['over_ground_distance'].apply(lambda x: float(x.replace(' nm', '')))
    df_not_in['play'] = df_not_in['play'].apply(lambda x: float(x.replace(' %', '')))
    
    # Clean data (df_in)
    df_in['rank'] = df_in['rank'].apply(lambda x: -1 if x == 'RET' else x)
    
    df_in['nationality'] = df_in['nationality & sail'].apply(lambda x: x.split()[0])
    df_in['sail'] = df_in['nationality & sail'].apply(lambda x: x.split()[1])
    
    df_in['skipper_firstname'] = df_in['skipper & crew'].apply(lambda x: x.split('\n')[0].split()[0])
    df_in['skipper_lastname'] = df_in['skipper & crew'].apply(lambda x: ' '.join(x.split('\n')[0].split()[1:]).upper().replace(' ', ''))
    df_in['crew_name'] = df_in['skipper & crew'].apply(lambda x: x.split('\n')[1])
    
    df_in['hour_fr'] = df_in['hour_fr'].apply(lambda x: datetime.datetime.strptime(x.split(' FR')[0], '%H:%M').strftime('%H:%M:%S') if isinstance(x, str) else np.nan)
    # df_in['latitude'] = df_in['latitude'].apply(lambda x: dms2dec(*x))
    # df_in['latitude'] = df_in['latitude'].apply(lambda x: ([int(number) for number in re.findall(r'\d+', x)]) if isinstance(x, str) else np.nan)
    
    df_in['since30min_heading'] = df_in['since30min_heading'].apply(lambda x: float(x.replace('°', '')) if isinstance(x, str) else np.nan)
    df_in['sincelastreport_heading'] = df_in['sincelastreport_heading'].apply(lambda x: float(x.replace('°', '')) if isinstance(x, str) else np.nan)
    df_in['since24hours_heading'] = df_in['since24hours_heading'].apply(lambda x: float(x.replace('°', '')) if isinstance(x, str) else np.nan)
    
    df_in['since30min_speed'] = df_in['since30min_speed'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    df_in['sincelastreport_speed'] = df_in['sincelastreport_speed'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    df_in['since24hours_speed'] = df_in['since24hours_speed'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    
    df_in['since30min_vmg'] = df_in['since30min_vmg'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    df_in['sincelastreport_vmg'] = df_in['sincelastreport_vmg'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    df_in['since24hours_vmg'] = df_in['since24hours_vmg'].apply(lambda x: float(x.replace(' kts', '')) if isinstance(x, str) else np.nan)
    
    df_in['since30min_distance'] = df_in['since30min_distance'].apply(lambda x: float(x.replace(' nm', '')) if isinstance(x, str) else np.nan)
    df_in['sincelastreport_distance'] = df_in['sincelastreport_distance'].apply(lambda x: float(x.replace(' nm', '')) if isinstance(x, str) else np.nan)
    df_in['since24hours_distance'] = df_in['since24hours_distance'].apply(lambda x: float(x.replace(' nm', '')) if isinstance(x, str) else np.nan)
    
    df_in['dtf'] = df_in['dtf'].apply(lambda x: float(x.replace(' nm', '')) if isinstance(x, str) else np.nan)
    df_in['dtl'] = df_in['dtl'].apply(lambda x: float(x.replace(' nm', '')) if isinstance(x, str) else np.nan)
    
    
    # Drop useless columns
    df_not_in = df_not_in.drop(columns=['nationality & sail', 'skipper & crew'])
    df_in = df_in.drop(columns=['nationality & sail', 'skipper & crew'])
    
    # Set ordered columns
    ordered_cols_not_in = ['ranking_datetime', 'skipper_status', 'rank', 'sail', 'nationality', 'skipper_lastname', 'skipper_firstname', 'crew_name', 'race_time', 'race_time_gap']
    df_not_in = df_not_in.reindex(columns=(ordered_cols_not_in + list([col for col in df_not_in.columns if col not in ordered_cols_not_in])))
    
    ordered_cols_in = ['ranking_datetime', 'skipper_status', 'rank', 'sail', 'nationality', 'skipper_lastname', 'skipper_firstname', 'crew_name']
    df_in = df_in.reindex(columns=(ordered_cols_in + list([col for col in df_in.columns if col not in ordered_cols_in])))
    
    # Replace '' by NaN
    df_not_in = df_not_in.replace('', np.nan)
    df_in = df_in.replace('', np.nan)
    
    return df_not_in, df_in

In [91]:
skippers_race_status_processed_dir_dst = DATA_PROCESSED_DIR + 'skippers_race_status/'

filename_not_in_race = skippers_race_status_processed_dir_dst + 'skippers_not_in_race.csv'
filename_in_race = skippers_race_status_processed_dir_dst + 'skippers_in_race.csv'

# Create folder if does not exist 
create_dir(skippers_race_status_processed_dir_dst)

# Check if directory is empty and retrieve boats characteristics
if not os.listdir(skippers_race_status_processed_dir_dst):
    print(f"Directory '{skippers_race_status_processed_dir_dst}' is empty. Cleaning skippers ranking..")

    # Preprocess data
    df_skippers_not_in_race, df_skippers_in_race = preprocess_skippers_ranking(df_skippers_not_in_race_raw, df_skippers_in_race_raw, verbose=True)
    print(f"Cleaning done.\n")
    
    # Save data
    save_csv(df_skippers_not_in_race, csv_path=filename_not_in_race, verbose=True)
    save_csv(df_skippers_in_race, csv_path=filename_in_race, verbose=True)
else:
    df_skippers_in_race = load_csv(filename_in_race)
    df_skippers_not_in_race = load_csv(filename_not_in_race)
    
df_skippers_not_in_race

# df_skippers_in_race[df_skippers_in_race['sail']=='222']#['Unnamed: 1'].unique()

,ranking_datetime,skipper_status,rank,sail,nationality,skipper_lastname,skipper_firstname,crew_name,race_time,race_time_gap,arrival_date,gaps_to_first,gaps_to_previous,over_ortho_speed,over_ortho_distance,play,over_ground_speed,over_ground_distance
0,2021-01-27 18:00:00,NOT IN RACE,1,79,FRA,DALIN,Charlie,APIVIA,80j 06h 15min 47s,NaN,2021-01-27 21:35:47,NaN,NaN,12.6,24355.0,119.6,15.1,29135.0
1,2021-01-27 22:00:00,NOT IN RACE,1,17,FRA,BESTAVEN,Yannick,Maître Coq IV,80j 03h 44min 46s,-10h 15min 00s,2021-01-28 05:19:46,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8
2,2021-01-27 22:00:00,NOT IN RACE,2,79,FRA,DALIN,Charlie,APIVIA,80j 06h 15min 47s,NaN,2021-01-27 21:35:47,02h 31min 01s,02h 31min 01s,12.6,24365.7,119.6,15.1,29135.0
3,2021-01-27 22:00:00,NOT IN RACE,3,18,FRA,BURTON,Louis,Bureau Vallée 2,80j 10h 25min 12s,NaN,2021-01-28 01:45:12,06h 40min 26s,04h 09min 25s,12.6,24365.7,117.6,14.8,28650.0
4,2021-01-28 05:00:00,NOT IN RACE,1,17,FRA,BESTAVEN,Yannick,Maître Coq IV,80j 03h 44min 46s,-10h 15min 00s,2021-01-28 05:19:46,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,2021-03-05 09:00:00,NOT IN RACE,21,83,FRA,GIRAUD,Clément,Compagnie du lit - Jiliti,99j 20h 08min 31s,NaN,2021-02-16 10:28:31,19j 16h 23min 45s,2j 13h 41min 28s,10.2,24365.7,115.5,11.7,28137.8
3927,2021-03-05 09:00:00,NOT IN RACE,22,50,FRA,MERRON,Miranda,Campagne de France,101j 08h 56min 51s,NaN,2021-02-17 23:16:51,21j 05h 12min 05s,1j 12h 48min 20s,10.0,24365.7,113.5,11.4,27656.2
3928,2021-03-05 09:00:00,NOT IN RACE,23,71,FRA,COUSIN,Manuel,Groupe Sétin,103j 18h 15min 40s,NaN,2021-02-20 08:35:40,23j 14h 30min 54s,2j 09h 18min 49s,9.8,24365.7,119.5,11.7,29115.7
3929,2021-03-05 09:00:00,NOT IN RACE,24,72,FRA,BARRIER,Alexia,TSE - 4myplanet,111j 17h 03min 44s,NaN,2021-02-28 07:23:44,31j 13h 18min 58s,7j 22h 48min 04s,9.1,24365.7,115.6,10.5,28170.7


In [92]:
df_skippers_in_race

,ranking_datetime,skipper_status,rank,sail,nationality,skipper_lastname,skipper_firstname,crew_name,hour_fr,latitude,longitude,since30min_heading,since30min_speed,since30min_vmg,since30min_distance,sincelastreport_heading,sincelastreport_speed,sincelastreport_vmg,sincelastreport_distance,since24hours_heading,since24hours_speed,since24hours_vmg,since24hours_distance,dtf,dtl
0,2020-11-08 16:00:00,IN RACE,1,18,FRA,BURTON,Louis,Bureau Vallée 2,15:30:00,46°24.46'N,01°50.48'W,241.0,17.7,17.5,0.3,357.0,0.0,0.0,2788.0,201.0,0.3,0.3,6.1,24293.9,0.0
1,2020-11-08 16:00:00,IN RACE,2,10,MON,HERRMANN,Boris,Seaexplorer - Yacht Club De Monaco,15:31:00,46°24.34'N,01°49.82'W,241.0,11.1,10.9,0.4,357.0,0.0,0.0,2787.9,196.0,0.3,0.2,6.0,24294.2,0.4
2,2020-11-08 16:00:00,IN RACE,3,8,FRA,BEYOU,Jérémie,Charal,15:30:00,46°24.91'N,01°49.99'W,244.0,15.5,15.5,0.5,357.0,0.0,0.0,2788.5,199.0,0.2,0.2,5.5,24294.3,0.5
3,2020-11-08 16:00:00,IN RACE,4,59,FRA,RUYANT,Thomas,LinkedOut,15:30:00,46°24.71'N,01°49.68'W,244.0,13.2,13.1,0.7,357.0,0.0,0.0,2788.3,196.0,0.2,0.2,5.6,24294.5,0.6
4,2020-11-08 16:00:00,IN RACE,5,53,FRA,SOREL,Maxime,V And B Mayenne,15:30:00,46°24.59'N,01°49.56'W,246.0,10.9,10.9,0.2,357.0,0.0,0.0,2788.1,195.0,0.8,0.7,5.8,24294.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19225,2021-03-05 09:00:00,IN RACE,-1,109,FRA,DAVIES,Samantha,Initiatives - Coeur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19226,2021-03-05 09:00:00,IN RACE,-1,4,FRA,SIMON,Sébastien,ARKEA PAPREC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19227,2021-03-05 09:00:00,IN RACE,-1,99,GBR,THOMSON,Alex,HUGO BOSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19228,2021-03-05 09:00:00,IN RACE,-1,85,FRA,ESCOFFIER,Kevin,PRB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
lat = df_skippers_in_race['latitude'].values[0]
print('lat', lat)

# lon = df_skippers_in_race['longitude'].values[0]
# print('lon', lon)

lat_split = lat.split('°')
deg = float(lat_split[0])
mn = float(lat_split[-1].split("'N")[0])
print(f"deg: {deg}, mn: {mn}")

dec = dms2dec(deg=deg, mn=mn, sec=0)
print('dec', dec)


lat 46°24.46'N
deg: 46.0, mn: 24.46
dec 46.407666666666664


In [146]:
df_skippers_in_race[df_skippers_in_race['latitude'].str.contains("'N")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

### B) Extraction des caractéristiques techniques de chacun des voiliers.

In [93]:
def convert_fr_date2en_date(fr_date):
    """ Convert french date into english date
    Args:
    Returns:
    """
    day, month, year = tuple(fr_date.split())
    month = month_name2month_no[unidecode(month).lower()]
    return datetime.datetime(int(year), int(month), int(day))


def preprocess_boats_characteristics(df_boats_characteristics, verbose=False):
    """ Load skippers ranking from xslx files into dataframes
    Args:
        df_boats_characteristics (DataFrame): data with skippers whos have finished the race
    Returns:
        df (DataFrame): preprocessed dataframe
    """
    df = df_boats_characteristics.copy()

    # Rename columns
    df = df.rename(columns={
        'skipper_name': 'skipper_name',
        'boat_name': 'crew_name',
        'Numéro de voile': 'nationality & sail',
        'Anciens noms du bateau': 'former_crew_names',
        'Architecte': 'architect',
        'Chantier': 'place_of_work',
        'Date de lancement': 'launch_date',
        'Longueur': 'length',
        'Largeur': 'width',
        "Tirant d'eau": 'water_draft',
        'Déplacement (poids)': 'weigth',
        'Nombre de dérives': 'drift_amount',  
        'Hauteur mât': 'mast_height',  
        'Voile quille': 'keel_sail',  
        'Surface de voiles au près': 'upwind_sail_area',  
        'Surface de voiles au portant': 'downwind_sail_area',    
    })

    # Skipper meta-data
    df['skipper_firstname'] = df['skipper_name'].apply(lambda x: ''.join([word for word in x.split() if not word.isupper()]))
    df['skipper_lastname'] = df['skipper_name'].apply(lambda x: unidecode(''.join([word for word in x.split() if word.isupper()])))
    df['nationality & sail'] = df['nationality & sail'].apply(lambda x: ' '.join([x[:3], x[3:]]) if (isinstance(x, str) and (len(x) > 3)) else x)
    df['nationality'] = df['nationality & sail'].apply(lambda x: ''.join([l for l in x if (l.lower().isalpha())]) if isinstance(x, str) else np.nan)
    df['sail'] = df['nationality & sail'].apply(lambda x: ''.join([l for l in x if l.isdigit()]) if isinstance(x, str) else np.nan)
    df = df.drop(columns=['skipper_name', 'nationality & sail'])
    df = df.reindex(columns=(['skipper_firstname', 'skipper_lastname', 'nationality', 'sail'] + list([col for col in df.columns if col not in ['skipper_firstname', 'skipper_lastname', 'nationality', 'sail']])))

    # Convert launch_date into datetime format
    df['launch_date'] = df['launch_date'].apply(lambda x: convert_fr_date2en_date(x))

    # Clean length, width, water_draft, weigth, mast_height, upwind_sail_area, downwind_sail_area
    df['length'] = df['length'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
    df['width'] = df['width'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
    df['water_draft'] = df['water_draft'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
    df['weigth'] = df['weigth'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
    df['mast_height'] = df['mast_height'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
    df['upwind_sail_area'] = df['upwind_sail_area'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)
    df['downwind_sail_area'] = df['downwind_sail_area'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)

    # Map drift_amount
    drift_amount2no = {drift_amount:no for no, drift_amount in enumerate(df['drift_amount'].dropna().unique().tolist())}
    no2drift_amounts = {v:k for k,v in drift_amount2no.items()}
    df['drift_amount'] = df['drift_amount'].apply(lambda x: drift_amount2no[x] if isinstance(x, str) else np.nan)

    # Map keel_sail
    keel2no = {keel:no for no, keel in enumerate(df['keel_sail'].dropna().unique().tolist())}
    no2keels = {v:k for k,v in keel2no.items()}
    df['keel_sail'] = df['keel_sail'].apply(lambda x: keel2no[x] if isinstance(x, str) else np.nan)
    
    # Handle outlier
    outlier_skipper_name = 'FrançoisGuiffant'
    outlier_skipper_name_idx = df[df['skipper_firstname']==outlier_skipper_name].index[0]
    df.at[outlier_skipper_name_idx, 'skipper_firstname'] = 'François'
    df.at[outlier_skipper_name_idx, 'skipper_lastname'] = 'Guiffant'.upper()
    
    # Replace '' by NaN
    df = df.replace('', np.nan)
    
    # Add skipper id
    df.insert(0, 'skipper_id', df.groupby(['skipper_lastname','skipper_firstname']).ngroup())

    return df

In [94]:
boat_processed_dir_dst = DATA_PROCESSED_DIR + 'boat/'

filename_raw = boat_raw_dir_dst + 'boats_characteristics.csv'
filename_processed = boat_processed_dir_dst + 'boats_characteristics.csv'

# Create folder if does not exist 
create_dir(boat_processed_dir_dst)

# Check if directory is empty and retrieve boats characteristics
if not os.listdir(boat_processed_dir_dst):
    print(f"Directory '{boat_processed_dir_dst}' is empty. Cleaning boats characteristics..")
    
    # Preprocess data
    df_boats_characteristics = preprocess_boats_characteristics(df_boats_characteristics_raw, verbose=True)
    print(f"Cleaning done.\n")
    
    # Save data
    save_csv(df_boats_characteristics, csv_path=filename_processed, verbose=True)
else:    
    df_boats_characteristics = load_csv(filename_processed)
    
df_boats_characteristics

,skipper_id,skipper_firstname,skipper_lastname,nationality,sail,crew_name,former_crew_names,architect,place_of_work,launch_date,length,width,water_draft,weigth,drift_amount,mast_height,keel_sail,upwind_sail_area,downwind_sail_area
0,0,Fabrice,AMEDEO,FRA,56.0,NEWREST - ART & FENÊTRES,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,2015-08-01,18.28,5.85,4.5,7.0,0.0,29.0,0.0,320.0,570.0
1,1,Romain,ATTANASIO,FRA,49.0,PURE - Best Western®,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),2007-03-08,18.28,5.80,4.5,9.0,1.0,28.0,1.0,280.0,560.0
2,2,Alexia,BARRIER,FRA,72.0,TSE - 4MYPLANET,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,1998-03-01,18.28,5.54,4.5,9.0,1.0,29.0,2.0,260.0,580.0
3,3,Yannick,BESTAVEN,NaN,17.0,Maître CoQ IV,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,2015-03-12,18.28,5.80,4.5,8.0,0.0,29.0,3.0,310.0,550.0
4,4,Jérémie,BEYOU,NaN,8.0,CHARAL,NaN,VPLP,CDK Technologies,2018-08-18,18.28,5.85,4.5,8.0,0.0,29.0,2.0,320.0,600.0
5,5,Arnaud,BOISSIERES,FRA,14.0,LA MIE CÂLINE - ARTISANS ARTIPÔLE,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,2007-08-03,18.28,5.65,4.5,7.9,0.0,29.0,4.0,300.0,610.0
6,6,Louis,BURTON,NaN,18.0,BUREAU VALLEE 2,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,2015-06-09,18.28,5.80,4.5,7.6,0.0,28.0,2.0,300.0,600.0
7,7,Didac,COSTA,ESP,33.0,ONE PLANET ONE OCEAN,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,2000-02-02,18.28,5.30,4.5,8.9,1.0,26.0,2.0,240.0,470.0
8,8,Manuel,COUSIN,FRA,71.0,GROUPE SÉTIN,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),2007-02-02,18.28,5.80,4.5,9.0,2.0,28.5,5.0,270.0,560.0
9,9,Clarisse,CREMER,FRA,30.0,BANQUE POPULAIRE X,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,2011-03-01,18.28,5.70,4.5,7.7,1.0,29.0,1.0,340.0,570.0


### C) NaN values analysis

In [95]:
def plot_NaN_values(df_plot, title, fig, plot_no, data_type):
    """ Visualize NaN values """
    df_nan_values = df_plot.isna().sum().to_frame(name='count')
    df_nan_values['percentage'] = round(df_nan_values['count']/df_plot.shape[0]*100)
    df_nan_values['percentage_name'] = df_nan_values['percentage'].astype(str) + "%"
    df_nan_values['data_type'] = df_nan_values.index
    df_nan_values = df_nan_values.reset_index(drop=True)
    df_nan_values = df_nan_values.sort_values(by=['percentage'])

    fig.add_trace(go.Bar(
        name=title,
        x=df_nan_values['count'],
        y=df_nan_values['data_type'],
        text=df_nan_values['percentage_name'],
        customdata=df_nan_values['percentage_name'],
        hovertemplate="<b>%{y}</b><br>Count: %{x}/"+str(df_plot.shape[0])+"<br>Percentage: %{customdata}",
        orientation='h'
    ),
    row=plot_no[0] , col=plot_no[1]
    )
    fig.update_traces(textfont_size=20, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_layout(
        height=1500,# width=1200,
        title={"text": f"<b>NaN values distribution ({data_type} data)</b>", "x": 0.5},
    )
    return fig



def subplot_NaN_values(df_skippers_not_in_race, df_skippers_in_race, df_boats_characteristics, subtitles, data_type='raw'):
    """ Subplot NaN values """
    
    N_rows, N_cols = 3, 1
    fig = make_subplots(
        rows=N_rows, cols=N_cols,
        subplot_titles=subtitles,
        vertical_spacing = 0.1
    )

    fig = plot_NaN_values(df_skippers_not_in_race, title=subtitles[0], fig=fig, plot_no=(1, 1), data_type=data_type)
    fig = plot_NaN_values(df_skippers_in_race, title=subtitles[1], fig=fig , plot_no=(2, 1), data_type=data_type)
    fig = plot_NaN_values(df_boats_characteristics, title=subtitles[2], fig=fig , plot_no=(3, 1), data_type=data_type)

    # Update xaxis properties
    fig.update_xaxes(title_text="NaN values count", range=[0, df_skippers_not_in_race.shape[0]], row=1, col=1)
    fig.update_xaxes(title_text="NaN values count", range=[0, df_skippers_in_race.shape[0]], row=2, col=1)
    fig.update_xaxes(title_text="NaN values count", range=[0, df_boats_characteristics.shape[0]], row=3, col=1)

    # Update xaxis properties
    fig.update_yaxes(title_text="Column name", row=1, col=1)
    fig.update_yaxes(title_text="Column name", row=2, col=1)
    fig.update_yaxes(title_text="Column name", row=3, col=1)

    fig.show()

In [96]:
subtitles = ['Skippers have finished the race', 'Skippers still in race/have abandonned', 'Boats characteristics']
subplot_NaN_values(df_skippers_not_in_race_raw, df_skippers_in_race_raw, df_boats_characteristics_raw, subtitles=subtitles, data_type='raw')

In [97]:
subtitles = ['Skippers have finished the race', 'Skippers still in race/have abandonned', 'Boats characteristics']
subplot_NaN_values(df_skippers_not_in_race, df_skippers_in_race, df_boats_characteristics, subtitles=subtitles, data_type='preprocessed')

### D) Rapprochement des données des voiliers avec celles des classements.

In [127]:
skipper_lastname2skipper_id = dict(map(reversed, df_boats_characteristics[['skipper_id', 'skipper_lastname']].drop_duplicates().values))
print(skipper_lastname2skipper_id)

{'AMEDEO': 0, 'ATTANASIO': 1, 'BARRIER': 2, 'BESTAVEN': 3, 'BEYOU': 4, 'BOISSIERES': 5, 'BURTON': 6, 'COSTA': 7, 'COUSIN': 8, 'CREMER': 9, 'DALIN': 10, 'DAVIES': 11, 'DESTREMAU': 12, 'DUTREUX': 13, 'ESCOFFIER': 14, 'GIRAUD': 15, 'GUIFFANT': 16, 'HARE': 17, 'HERRMANN': 18, 'HUUSELA': 19, 'JOSCHKE': 20, 'LECAM': 21, 'LEDIRAISON': 22, 'MERRON': 23, 'PEDOTE': 24, 'ROURA': 25, 'RUYANT': 26, 'SEGUIN': 27, 'SHIRAISHI': 28, 'SIMON': 29, 'SOREL': 30, 'THOMSON': 31, 'TRIPON': 32, 'TROUSSEL': 33}


In [99]:
unique_skippers_boats = list(sorted(skipper_lastname2skipper_id.keys()))
print(f"unique_skippers_boats ({len(unique_skippers_boats)}):\n{unique_skippers_boats}\n")

unique_skippers_not_in = list(sorted(df_skippers_not_in_race['skipper_lastname'].unique().tolist()))
print(f"unique_skippers_not_in ({len(unique_skippers_not_in)}):\n{unique_skippers_not_in}\n")

unique_skippers_in = list(sorted(df_skippers_in_race['skipper_lastname'].unique().tolist()))
print(f"unique_skippers_in ({len(unique_skippers_in)}):\n{unique_skippers_in}")

unique_skippers_boats (34):
['AMEDEO', 'ATTANASIO', 'BARRIER', 'BESTAVEN', 'BEYOU', 'BOISSIERES', 'BURTON', 'COSTA', 'COUSIN', 'CREMER', 'DALIN', 'DAVIES', 'DESTREMAU', 'DUTREUX', 'ESCOFFIER', 'GIRAUD', 'GUIFFANT', 'HARE', 'HERRMANN', 'HUUSELA', 'JOSCHKE', 'LECAM', 'LEDIRAISON', 'MERRON', 'PEDOTE', 'ROURA', 'RUYANT', 'SEGUIN', 'SHIRAISHI', 'SIMON', 'SOREL', 'THOMSON', 'TRIPON', 'TROUSSEL']

unique_skippers_not_in (25):
['ATTANASIO', 'BARRIER', 'BESTAVEN', 'BEYOU', 'BOISSIERES', 'BURTON', 'COSTA', 'COUSIN', 'CREMER', 'DALIN', 'DUTREUX', 'GIRAUD', 'HARE', 'HERRMANN', 'HUUSELA', 'LECAM', 'LEDIRAISON', 'MERRON', 'PEDOTE', 'ROURA', 'RUYANT', 'SEGUIN', 'SHIRAISHI', 'SOREL', 'TRIPON']

unique_skippers_in (33):
['AMEDEO', 'ATTANASIO', 'BARRIER', 'BESTAVEN', 'BEYOU', 'BOISSIERES', 'BURTON', 'COSTA', 'COUSIN', 'CREMER', 'DALIN', 'DAVIES', 'DESTREMAU', 'DUTREUX', 'ESCOFFIER', 'GIRAUD', 'HARE', 'HERRMANN', 'HUUSELA', 'JOSCHKE', 'LECAM', 'LEDIRAISON', 'MERRON', 'PEDOTE', 'ROURA', 'RUYANT', 'SEGUIN'

In [100]:
# Add skipper id
if 'skipper_id' not in df_skippers_not_in_race.columns:
    df_skippers_not_in_race['skipper_id'] = df_skippers_not_in_race['skipper_lastname'].apply(lambda x: skipper_lastname2skipper_id[x])
    df_skippers_not_in_race = df_skippers_not_in_race.reindex(columns=(['ranking_datetime', 'skipper_id',] + list([col for col in df_skippers_not_in_race.columns if col not in ['ranking_datetime', 'skipper_id']])))

if 'skipper_id' not in df_skippers_in_race.columns:
    df_skippers_in_race['skipper_id'] = df_skippers_in_race['skipper_lastname'].apply(lambda x: skipper_lastname2skipper_id[x])
    df_skippers_in_race = df_skippers_in_race.reindex(columns=(['ranking_datetime', 'skipper_id',] + list([col for col in df_skippers_in_race.columns if col not in ['ranking_datetime', 'skipper_id']])))

# Partie 3 - Analyses et story telling


### A) Classement des skippers au cours du temps

In [128]:
cols = ['skipper_id', 'ranking_datetime', 'skipper_status', 'rank', 'skipper_lastname', 'skipper_firstname']
df_timeline = pd.concat([df_skippers_not_in_race[cols], df_skippers_in_race[cols]], ignore_index=True)
print(f"Total: df_timeline.shape: {df_timeline.shape[0]}\n\t- df_skippers_not_in_race.shape: {df_skippers_not_in_race.shape[0]}\n\t- df_skippers_in_race.shape: {df_skippers_in_race.shape[0]}")

# Setting the index as a datetime object
df_timeline = df_timeline.set_index('ranking_datetime')
df_timeline = df_timeline.sort_index(ascending=False)
df_timeline.index = pd.to_datetime(df_timeline.index)

N_skippers = df_timeline['skipper_id'].max() + 1
print(f"N_skippers: {N_skippers}")
df_timeline['rank'] = df_timeline['rank'].apply(lambda x: N_skippers if x == -1 else x)
df_timeline

Total: df_timeline.shape: 23161
	- df_skippers_not_in_race.shape: 3931
	- df_skippers_in_race.shape: 19230
N_skippers: 34


,skipper_id,skipper_status,rank,skipper_lastname,skipper_firstname
ranking_datetime,,,,,
2021-03-05 09:00:00,33,IN RACE,34,TROUSSEL,Nicolas
2021-03-05 09:00:00,23,NOT IN RACE,22,MERRON,Miranda
2021-03-05 09:00:00,9,NOT IN RACE,12,CREMER,Clarisse
2021-03-05 09:00:00,4,NOT IN RACE,13,BEYOU,Jérémie
2021-03-05 09:00:00,1,NOT IN RACE,14,ATTANASIO,Romain
...,...,...,...,...,...
2020-11-08 16:00:00,20,IN RACE,28,JOSCHKE,Isabelle
2020-11-08 16:00:00,29,IN RACE,29,SIMON,Sébastien
2020-11-08 16:00:00,23,IN RACE,30,MERRON,Miranda


In [131]:
def get_skipper_timeline(df, skipper_id):
    """ Get skipper timeline for a specific skipper """
    df_skipper_timeline = df[df['skipper_id']==skipper_id]
    return df_skipper_timeline


def generate_color_categories(categories, rgb=True):
    """ Generate n colors, useful for coloring different categories 
    Args:
        categories (Array of strings): categories to process
    Returns:
        color_map (Dictionary): contains color for each category
    """
    ncol = len(categories)
    if ncol <= 50:
        colors = [i for i in get_cmap('tab20b').colors]
        colors += [i for i in get_cmap('tab20c').colors]
        colors += [i for i in get_cmap('tab20c_r').colors]
        colors = colors[:ncol]
    elif 50 < ncol <= 256:
        cmap = get_cmap(name='viridis')
        cmap = get_cmap(name='rainbow')
        colors = cmap(np.linspace(0, 1, ncol))
    else:
        raise ValueError('Maximum 256 categories')
    
    color_map = {}
    for i, key in enumerate(categories):
        if rgb:
            color_map[key] = 'rgb({r},{g},{b})'.format(r=colors[i][0], g=colors[i][1], b=colors[i][2])
        else:
            color_map[key] = colors[i][:3]
    return color_map


def display_ranking_timeline(df_timeline):
    """ Get skipper timeline for a specific skipper """
    colors = generate_color_categories(df_timeline['skipper_id'].unique())
    skipper_ids = sorted(df_timeline['skipper_id'].unique().tolist())
    
    fig = go.Figure()
    for skipper_id in skipper_ids:
        df_skipper_timeline = get_skipper_timeline(df_timeline, skipper_id)
        fig.add_trace(
            go.Scatter(
                x=df_skipper_timeline.index, 
                y=df_skipper_timeline['rank'],
                mode='markers+lines',
                name=f"{df_skipper_timeline['skipper_lastname'].values[0]} {df_skipper_timeline['skipper_firstname'].values[0]}",
                marker_color=colors[skipper_id]
            )
        )

    fig.update_traces(marker_line_width=1, marker_size=8)
    fig.update_layout(
        width=1500, height=800, 
        title=dict(text=f"<b>Skippers ranking timeline</b>", x=.5),
        legend_title_text='Skipper name',
    
        xaxis=dict(title='Timeline', side='left', rangeslider=dict(visible=True, thickness=0.15)),
        yaxis=dict(title='Ranking', autorange='reversed', tickmode='array', tickvals=list(range(1, int(df_timeline['rank'].max())+1)))
    )
    fig.show()

In [132]:
display_ranking_timeline(df_timeline)

In [ ]:
# def create_count_df(df, feature_name=None, index=False):
#     """ Create a dataframe with the feature count 
#     Args:
#         df (DataFrame): data to process
#         feature_name (String): feature name to count
#     Returns:
#         df_feature (DataFrame): data with feature count
#     """
#     if index:
#         df_feature = df.index.value_counts().to_frame(name='count')
#         feature_name = 'index'
#     else:
#         df_feature = df[feature_name].value_counts().to_frame(name='count')
#     df_feature.insert(0, feature_name, df_feature.index)  
#     df_feature = df_feature.sort_values(by=['count'], ascending=False)
#     df_feature = df_feature.reset_index(drop=True)
#     return df_feature


# # Count index time
# df_plot = create_count_df(df_timeline, index=True)

# fig = px.bar(df_plot, y='count')
# fig.update_layout(
#     width=1200, height=600, 
#     title=dict(
#         text=f"Nombre de classements pau cours du temps",
#         x=.5,
#         font_size=18,
#     )
# )
# fig.show()

# df_plot

### B) Corrélation et régression linéaire entre le classement (rang) et la vitesse utile (VMG) des voiliers.

In [ ]:
df_skippers_not_in_race.head(3)

In [ ]:
df_skippers_in_race.head(3)

In [ ]:
df_boats_characteristics.head(3)

In [ ]:
boats_useful_cols = list(df_boats_characteristics.columns)[5:]
print(f"boats_useful_cols: {boats_useful_cols}")

df_skippers_boats = df_skippers_not_in_race.copy()
for col in boats_useful_cols:
    df_skippers_boats[col] = df_skippers_boats['skipper_id'].apply(lambda x: df_boats_characteristics[df_boats_characteristics['skipper_id']==x][col].values[0])
df_skippers_boats

In [ ]:
df_skippers_boats['rank'].unique()

In [ ]:
df_corr = df_skippers_boats.copy()
# df_corr = df_corr.dropna()
# df_corr['length'] = df_corr['length'].astype(int)
corr_matrix = df_corr.corr()

fig, ax = plt.subplots(figsize=(15, 10))  
sns.heatmap(corr_matrix, annot=True, linewidths=.5, fmt=".2f", ax=ax)
plt.title("Correlation matrix\n")
plt.show()

### C) Impact de la présence d'un foil sur le classement et la vitesse des voiliers.

### D) Visualisation de la distance parcourue par voilier.

### E) Cartes avec les routes d'un ou plusieurs voiliers.

### F) Analyses de séries temporelles.

### G) Application d'algorithmes statistiques ou de machine learning.

### H) etc.

In [ ]:
# remove_dir(ranking_dir_dst)
# remove_dir(uuid_dir_dst)

# remove_dir(skippers_race_status_raw_dir_dst)
# remove_dir(skippers_race_status_preprocessed_dir_dst)

# remove_dir(boat_raw_dir_dst)
# remove_dir(boat_preprocessed_dir_dst)